# VNP46A2 Quality Flag Bands Exploration

Explores the Mandatory_Quality_Flag, Snow_Flag, and QF_Cloud_Mask bands in a VNP46A2 image.

The tables in this Notebook are found in the [Black Marble User Guide Version 1.0](https://viirsland.gsfc.nasa.gov/PDF/VIIRS_BlackMarble_UserGuide.pdf).

**QF_Cloud_Mask (base-10) (Adapted from User Guide p. 14):**

| Bit | Flag Description Key                          | Interpretation                                                                            |
|:-----|:-----------------------------------------------|:-------------------------------------------------------------------------------------------|
| 0   | Day/Night                                     | 0 = Night <br> 1 = Day                                                                         |
| 1-3 | Land/Water Background                         | 0 = Land & Desert <br> 1 = Land no Desert <br> 2 = Inland Water <br> 3 = Sea Water <br> 5 = Coastal |
| 4-5 | Cloud Mask Quality                            | 0 = Poor <br> 1 = Low <br> 2 = Medium <br> 3 = High                                                  |
| 6-7 | Cloud Detection Results & Confidence Indicator | 0 = Confident Clear <br> 1 = Probably Clear <br> 2 = Probably Cloudy <br> 3 = Confident Cloudy     |
| 8   | Shadow Detected                               | 0 = No <br> 1 = Yes                                                                             |
| 9   | Cirrus Detection (IR) (BTM15 –BTM16)          | 0 = No Cloud <br> 1 = Cloud                                                                   |
| 10  | Snow/Ice Surface                              | 0 = No Snow/Ice <br> 1 = Snow/Ice     |

<br>

**Mandatory_Cloud_Flag (base-10) (User Guide p. 16):**

| Value | Retrieval Quality | Algorithm Instance                                                      |
|:-------|:-------------------|:-------------------------------------------------------------------------|
| 0     | High-quality      | Main algorithm (Persistent nighttime lights)                            |
| 1     | High-quality      | Main algorithm (Ephemeral Nighttime Lights)                             |
| 2     | Poor-quality      | Main algorithm (Outlier, potential cloud contamination or other issues) |
| 255   | No retrieval      | Fill value                  |




**Snow_Flag (base-10) (User Guide p. 16)**:

| Flag Description Key | Value         | Interpretation                        |
|:----------------------|:---------------|:---------------------------------------|
| Snow/Ice Surface     | 0<br>1<br>255 | No Snow/Ice<br>Snow/Ice<br>Fill Value |

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import warnings
import numpy as np
import earthpy.plot as ep
import viirs

In [ ]:
# Set options
warnings.filterwarnings("ignore")

In [ ]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

# Data Acquisition and Preprocessing

In [ ]:
# Set path to VNP46A2 test image (Jun 6, 2016)
hdf5_path = os.path.join(
    "02-raw-data",
    "hdf",
    "south-korea",
    "vnp46a2",
    "VNP46A2.A2016153.h30v05.001.2020267141459.h5",
)

In [ ]:
# Extract all VNP46A2 bands
dnb_brdf_corrected_ntl = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="DNB_BRDF-Corrected_NTL"
)
dnb_lunar_irradiance = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="DNB_Lunar_Irradiance"
)
gap_filled_dnb_brdf_corrected_ntl = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="Gap_Filled_DNB_BRDF-Corrected_NTL"
)
latest_high_quality_retrieval = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="Latest_High_Quality_Retrieval"
)
mandatory_quality_flag = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="Mandatory_Quality_Flag"
)
qf_cloud_mask = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="QF_Cloud_Mask"
)
snow_flag = viirs.extract_band_vnp46a2(
    hdf5_path=hdf5_path, band_name="Snow_Flag"
)

In [ ]:
# Show main bands unique values (within the single image)
print(
    (
        "Latest High Quality Retrieval: "
        f"{viirs.get_unique_values(latest_high_quality_retrieval)}"
    )
)
print(
    (
        f"Mandatory Quality Flag: "
        f"{viirs.get_unique_values(mandatory_quality_flag)}"
    )
)
print(f"Snow Flag: {viirs.get_unique_values(snow_flag)}")

In [ ]:
# Extract QF Cloud Mask bitmasks
day_night = viirs.extract_qa_bits(qf_cloud_mask, 0, 0)
land_water_background = viirs.extract_qa_bits(qf_cloud_mask, 1, 3)
cloud_mask_quality = viirs.extract_qa_bits(qf_cloud_mask, 4, 5)
cloud_detection = viirs.extract_qa_bits(qf_cloud_mask, 6, 7)
shadow_detected = viirs.extract_qa_bits(qf_cloud_mask, 8, 8)
cirrus_detection = viirs.extract_qa_bits(qf_cloud_mask, 9, 9)
snow_ice_surface = viirs.extract_qa_bits(qf_cloud_mask, 10, 10)

In [ ]:
# Show QF Cloud Mask bitmask unique values (within the single image)
print(f"Day/Night: {viirs.get_unique_values(day_night)}")
print(
    f"Land/Water Background: {viirs.get_unique_values(land_water_background)}"
)
print(f"Cloud Mask Quality: {viirs.get_unique_values(cloud_mask_quality)}")
print(f"Coud Detection Results: {viirs.get_unique_values(cloud_detection)}")
print(f"Shadow Detected: {viirs.get_unique_values(shadow_detected)}")
print(f"Cirrus Detection: {viirs.get_unique_values(cirrus_detection)}")
print(f"Snow/Ice Surface: {viirs.get_unique_values(snow_ice_surface)}")

In [ ]:
# Create quality flag stack
quality_flag_stack = viirs.stack_quality_flags_vnp46a2(vnp46a2_path=hdf5_path)
print(
    f"Quality stack shape (bands, rows, columns): {quality_flag_stack.shape}"
)

# Data Processing

# Data Postprocessing

# Data Visualization

## Radiance Image

In [ ]:
# Plot raw DNB_BRDF-Corrected_NTL (before preprocessing)
radiance = ep.plot_bands(dnb_brdf_corrected_ntl, vmax=100)

## Single QA Bands

In [ ]:
# Plot mandatory quality flag
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=mandatory_quality_flag, bitmask_name="Mandatory Quality Flag"
)

In [ ]:
# Plot snow flag
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=snow_flag, bitmask_name="Snow Flag"
)

In [ ]:
# Plot day/night band
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=day_night, bitmask_name="Day/Night"
)

In [ ]:
# Plot land/water background
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=land_water_background, bitmask_name="Land/Water Background"
)

In [ ]:
# Plot cloud mask quality
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=cloud_mask_quality, bitmask_name="Cloud Mask Quality"
)

In [ ]:
# Plot cloud detection results
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=cloud_detection, bitmask_name="Cloud Detection"
)

In [ ]:
# Plot shadow detected
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=shadow_detected, bitmask_name="Shadow Detected"
)

In [ ]:
# Plot cirrus detection
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=shadow_detected, bitmask_name="Cirrus Detection"
)

In [ ]:
# Plot snow/ice surface
fig, ax = viirs.plot_quality_flag_bitmask_single_band_vnp46a2(
    bitmask_array=shadow_detected, bitmask_name="Snow/Ice Surface"
)

## All QA Bands

In [ ]:
# Plot all QA bands
fig, ax = viirs.plot_quality_flags_vnp46a2(
    vnp46a2_quality_stack=quality_flag_stack, data_source="NASA"
)

# Data Export